In [ ]:
!pip install simpletransformers

In [ ]:
%cd transformers

In [ ]:
%cd /mnt/batch/tasks/shared/LS_root/mounts/clusters/gptcomputeinstance/code/users/atherfawaz

In [ ]:
!python generate.py

In [13]:
!python data_prep.py

In [ ]:
!git clone https://github.com/NVIDIA/apex

In [ ]:
%cd apex

In [ ]:
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" --user ./

In [ ]:
!pip install -v --no-cache-dir ./

In [ ]:
%%writefile setup.sh

pip install -v --no-cache-dir ./

In [ ]:
%cd simpletransformers/examples/language_generation

In [ ]:
%cd apex

In [ ]:
!rm -rf build

In [ ]:
!nvcc --version 

In [ ]:
!pip install nvcc

In [ ]:
!pip install torch==1.4+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!python setup.py install --cuda_ext --cpp_ext 

In [ ]:
!pip show apex

In [ ]:
%cd simpletransformers/examples/language_generation

In [ ]:
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch -y

In [ ]:
!conda update -n base -c defaults conda -y

In [ ]:
!pip install torch

In [ ]:
!python fine_tune.py

In [12]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/gptcomputeinstance/code/users/atherfawaz/simpletransformers/examples/language_generation


In [14]:
!python fine_tune.py

INFO:simpletransformers.language_modeling.language_modeling_utils: Creating features from dataset file at cache_dir/
100%|████████████████████████████████████| 1118/1118 [00:00<00:00, 67155.00it/s]
INFO:simpletransformers.language_modeling.language_modeling_utils: Saving features into cached file cache_dir/gpt2_cached_lm_508_train.txt
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scal

In [16]:
!python generate.py

["Despite the recent successes of deep learning, such models are still far from some human abilities like learning from few examples, reasoning and explaining decisions. In this paper, we focus on organ annotation in medical images and we introduce a reasoning framework that is based on learning fuzzy relations on a small dataset for generating explanations. At least, that is what we mean. I would like to thank the many people who support our work and to suggest new problems that could lead to new discoveries. We also want to ask you what your feedback is. We can also find and fix bug reports by posting on this website, but I really don't want to give out your own suggestions. Any feedback is welcome!    urlLink     urlLink    I've noticed that many of the questions in my question are completely different from the ones I was asking. What is the most important difference?    urlLink    urlLink  I'm a neuroscientist. I work with"]
['There is a growing interest and literature on intrinsic